# 단어 문서 행렬 (Term Document Matrix)
- 문서별로 단어의 빈도(term frequency)를 정리한 표
 - 단어가 컬럼, 문서가 열이 됨
<br><br>
- 문서1: 오늘은 밥을 먹었다
- 문서2: 어제도 밥, 오늘도 밥

||오늘|어제|밥|먹다|
|:---|:---|:---|:---|:---|
|문서1|1|0|1|1|
|문서2|1|1|2|0|

## TDM의 장단점
- 비정형 데이터인 텍스트를 표 형태로 정형화
 - 정형 데이터의 다양한 통계 기법을 적용 가능
- 처리가 단순 ←→ 어순과 맥락을 무시하는 것이 단점

## 희소 행렬 (Sparse Matrix)
- 단어 문서 행렬에서 대부분의 값은 0
 - 의미있는 값이 있는 부분이 매우 드물다
- 이러한 행렬을 희소(sparse) 행렬이라 함 ←→ 조밀(dense)
- 용량을 아끼기 위해 0을 빼고 저장하는 압축 방법을 사용
 - 일반적인 표나 행렬처럼 계산하기 불편하므로 압축을 풀어야 함

---
## TDM 실습
### 데이터 열기

In [1]:
import pandas as pd

`index_col=0`: 파일에서 0번 컬럼이 행 번호를 나타냄을 뜻함

In [2]:
df = pd.read_excel('data/imdb.xlsx', index_col=0)
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [3]:
df.shape # 표의 형태 확인

(748, 2)

748행 2열

### TDM 만들기
`scikit-learn` 라이브러리에서 단어문서행렬을 만드는 `CountVectorizer` 모듈을 사용

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

#### `CountVectorizer`
: 토큰이 문서별로 몇 번(count) 등장했는지 행렬(vector; 표의 한 행)로 정리한다. 특별히 토큰화 방법을 지정하지 않으면, 빈 칸을 기준으로 토큰을 구분한다.

- `max_features`
 - 단어문서행렬에 포함시킬 최대(max)의 단어(feature) 수
 - `max_features=500`: 빈도 순으로 최대 500 단어까지 포함
- `stop_words`
 - 분석에서 제외할 불용어를 설정하는 옵션
 - `stop_words='english'`로 설정하면 영어의 경우 관사, 전치사 등을 제외
 - 다른 언어는 리스트 등의 형태로 불용어 목록을 넘겨주어야 함

In [5]:
# CountVectorizer 초기화
cv = CountVectorizer(max_features=500, stop_words='english')

In [6]:
# 단어문서행렬 만들기
tdm = cv.fit_transform(df['review'])

In [7]:
tdm

<748x500 sparse matrix of type '<class 'numpy.int64'>'
	with 3433 stored elements in Compressed Sparse Row format>

압축된 상태여서 바로 확인할 수 없음

### 단어 목록
단어 목록이 tdm이 아닌 cv에 저장되는 것에 주의

In [8]:
cv.get_feature_names()[:10]

['10',
 '20',
 '90',
 'ability',
 'absolutely',
 'acting',
 'action',
 'actor',
 'actors',
 'actress']

### 단어별 총빈도
||acted|actor|actors|
|:---|:---:|:---:|:---:|
|0|1|0|2|
|1|2|3|5|
||3|3|7|

`.sum()`: 합계
- `axis=0`: 열별 합계
- `axis=1`: 행별 합계

In [9]:
tdm.sum() # 표 전체의 총합계

3894

In [10]:
tdm.sum(axis=0) # 단어별 총빈도

matrix([[ 29,   3,   6,   3,   9,  43,   7,  10,  19,   3,   3,  10,   3,
           4,   3,   4,   9,   3,   3,   3,   6,   3,   4,  13,   4,   3,
           5,   8,   5,   3,  14,  71,   4,  11,   4,   6,   8,  25,  18,
          10,   3,   5,  10,   4,   3,   4,  10,   3,   3,   6,   7,   3,
           4,  10,   5,   3,  18,   6,   8,  24,  35,   4,   7,   4,   5,
           3,   8,  10,   6,   5,   3,   5,   7,   4,   3,   3,   5,   3,
           3,   4,   4,   7,   4,   5,   3,   4,   6,   4,   5,   4,   3,
           5,   4,   9,   4,   3,  13,  11,  22,   5,   3,   9,   5,  12,
           6,   3,   9,  12,  26,   4,  10,   3,   3,   3,   3,   4,   4,
           3,   6,   3,   9,   3,   3,  11,  11,   4,   5,   3,   9,   4,
           6,   3,   8,   3,   3,  16,   4,   7,   4,   4,   5,   8,   5,
           7,   3,   5,   7,   5,   3,   3,   3,   3,   3,  10,   5, 163,
          24,   3,   5,   6,   4,   3,   3,   3,   5,  19,   3,   9,   3,
           7,   5,   3,   4,   3,   6,

In [11]:
tdm.sum(axis=1) # 문서별 총 단어 수

matrix([[  5],
        [  4],
        [ 12],
        [  2],
        [  5],
        [  6],
        [  2],
        [  7],
        [  2],
        [  2],
        [  1],
        [  4],
        [  3],
        [  1],
        [  4],
        [  3],
        [  6],
        [ 10],
        [  4],
        [228],
        [  4],
        [  2],
        [  4],
        [  2],
        [  1],
        [  2],
        [  2],
        [  2],
        [  6],
        [  2],
        [  1],
        [  0],
        [  1],
        [  3],
        [  4],
        [  5],
        [  3],
        [  5],
        [  9],
        [  3],
        [  5],
        [  3],
        [  2],
        [  0],
        [  0],
        [  7],
        [  4],
        [  3],
        [  1],
        [  8],
        [  4],
        [  5],
        [  3],
        [  2],
        [  4],
        [  4],
        [  4],
        [  0],
        [  2],
        [  4],
        [  3],
        [  4],
        [  0],
        [  1],
        [  2],
        [  3],
        [ 

#### 데이터프레임으로 저장
- `.sum`의 결과가 행렬 형태이므로, `.flat`을 사용해 컬럼에 들어갈 수 있는 형태로 변환
- `sort_values()`: 정렬
 - `ascending=TRUE`: 오름차순 정렬
 - `ascending=False`: 내림차순 정렬

In [12]:
word_count = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm.sum(axis=0).flat
})
word_count.sort_values('빈도', ascending=False)

,단어,빈도
281,movie,182
155,film,163
31,bad,71
227,just,63
181,good,58
...,...,...
175,giallo,3
173,genuine,3
255,lots,3
291,obviously,3


### 단어 빈도 저장

In [13]:
word_count.to_csv('data/word_count.csv')